In [1]:

import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm

In [2]:
import pandas as pd
# 훈련 데이터와 테스트 데이터를 나누기 위해 사용한다
from sklearn import model_selection

df = pd.read_csv("ml-20m/ratings.csv")
# X는 (userId, movieId) 쌍
X = df[["userId", "movieId"]].values
Y = df[["rating"]].values

# 훈련 데이터와 테스트 데이터를 9대 1로 분할
train_X, test_X, train_Y, test_Y\
    = model_selection.train_test_split(X, Y, test_size=0.1)

# X는 ID이고 정수이므로 int64, Y는 실수이므로 float32의 Tensor로 변환
train_dataset = TensorDataset(
    torch.tensor(train_X, dtype=torch.int64), torch.tensor(train_Y, dtype=torch.float32))
test_dataset = TensorDataset(
    torch.tensor(test_X, dtype=torch.int64), torch.tensor(test_Y, dtype=torch.float32))
train_loader = DataLoader(
    train_dataset, batch_size=1024, num_workers=4, shuffle=True)
test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=4)

### 파이토치에서 행렬 인수분해

#### 행렬 인수분해

In [3]:
class MatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item, k=20):
        super().__init__()
        self.max_user = max_user
        self.max_item = max_item
        self.user_emb = nn.Embedding(max_user, k, 0)
        self.item_emb = nn.Embedding(max_item, k, 0)
        
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        
        # user_feature*item_feature는 (batch_size,k) 차원이므로
        # k의 sum을 구하면 각 샘플의 내적이 된다
        out = torch.sum(user_feature * item_feature, 1)
        
        # [0, 5] 범위 내로 조정
        out = nn.functional.sigmoid(out) * 5
        return out

In [4]:

max_user, max_item = X.max(0)
# np.int64형을 파이썬의 표준 int로 캐스트
max_user = int(max_user)
max_item = int(max_item)
net = MatrixFactorization(max_user+1, max_item+1)

In [5]:
def eval_net(net, loader, score_fn=nn.functional.
l1_loss, device="cpu"):
    ys = []
    ypreds = []
    for x, y in loader:
        x = x.to(device)
        ys.append(y)
        with torch.no_grad():
         ypred = net(x).to("cpu").view(-1)
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), 
torch.cat(ypreds))
    return score.item()

In [6]:
from statistics import mean

net.to("cuda:0")
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

for epoch in range(5):
    loss_log = []
    for x, y in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        o = net(x)
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score, flush=True)

  0%|          | 0/17579 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 17579/17579 [02:50<00:00, 103.31it/s]


0 1.6028930388373641 0.7323529720306396


100%|██████████| 17579/17579 [02:54<00:00, 100.98it/s]


1 0.8788287702447095 0.7079219222068787


100%|██████████| 17579/17579 [02:51<00:00, 102.80it/s]


2 0.8356876329686957 0.7009750604629517


100%|██████████| 17579/17579 [02:51<00:00, 102.67it/s]


3 0.8146483982522792 0.6964438557624817


100%|██████████| 17579/17579 [02:53<00:00, 101.22it/s]


4 0.802132907523536 0.6933056116104126


In [7]:
torch.save(net.state_dict(), '../model.pt')

In [8]:
# 훈련한 모델을 CPU로 이동
net.to("cpu")

# 사용자1의 영화 10에 대한 평가 계산
query = (1, 10)

# int64 Tensor로 변환하고 batch 차원을 추가
query = torch.tensor(query, dtype=torch.int64).view(1, -1)

# net에 전달
net(query)

tensor([3.7356], grad_fn=<MulBackward0>)

In [9]:
query = torch.stack([
    torch.zeros(max_item).fill_(1),
    torch.arange(1, max_item+1)
], 1).long()

# scores는 상위 k개의 점수
# indices는 상위 k개의 위치, 즉 movieId
scores, indices = torch.topk(net(query), 5)

In [12]:
class NeuralMatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item,
                 user_k=10, item_k=10,
                 hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            nn.Linear(user_k + item_k, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
            
    def forward(self, x):
        user_idx = x[:, 0]
        print('user_idx :',user_idx)
        item_idx = x[:, 1]
        print('item_idx :',item_idx)
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        # 사용자 특이량과 상품 특이량을 모아서 하나의 벡터로 만듦
        out = torch.cat([user_feature, item_feature], 1)
        # 모은 벡터를 MLP에 넣는다
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()

In [13]:
import csv
from sklearn.feature_extraction.text import CountVectorizer

# csv.DictReader를 사용해서 CSV 파일 읽기
# 필요한 부분만 추출
with open("ml-20m/movies.csv") as fp:
    reader = csv.DictReader(fp)
    def parse(d):
        movieId = int(d["movieId"])
        genres = d["genres"]
        return movieId, genres
    data = [parse(d) for d in reader]
  
movieIds = [x[0] for x in data]
genres = [x[1] for x in data]

# 데이터에 맞추어 CountVectorizer를 훈련
cv = CountVectorizer(dtype="f4").fit(genres)
num_genres = len(cv.get_feature_names())

# key가 movieId이고 value가 BoW인 Tensor의 dict 만들기
it = cv.transform(genres).toarray()
it = (torch.tensor(g, dtype=torch.float32) for g in it)
genre_dict = dict(zip(movieIds, it))

/home/innerwave/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
def first(xs):
    it = iter(xs)
    return next(it)

class MovieLensDataset(Dataset):
    def __init__(self, x, y, genres):
        assert len(x) == len(y)
        self.x = x
        self.y = y
        self.genres = genres
        
        # 장르 사전에 없는 movieId를 위한 더미 데이터
        self.null_genre = torch.zeros_like(
            first(genres.values()))
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        # x = (userId, movieId)
        movieId = x[1]
        g = self.genres.get(movieId, self.null_genre)
        return x, y, g

In [ ]:
train_dataset = MovieLensDataset(
    torch.tensor(train_X, dtype=torch.int64),
    torch.tensor(train_Y, dtype=torch.float32), 
    genre_dict)
test_dataset = MovieLensDataset(
    torch.tensor(test_X, dtype=torch.int64),
    torch.tensor(test_Y, dtype=torch.float32),
    genre_dict)
train_loader = DataLoader(
    train_dataset, batch_size=1024, shuffle=True, 
num_workers=4)
test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=4)